# 기본 MODULE 불러오기 

# Project_ GOAL: 
#포항시 일대 22~23년 분실데이터; 
(target: 분실 물품 유형별 잃어버린 장소 연관 학습모델: (버스, 기차, 택시 ) 
#(대중교통, 공공기관 시설) <-> 시설이 아닌지역  나누어 분석하여 입력값으로 물품 종류를 입력하면 해당 물품의 분실장소 예측, 분실 문의할수있는 연관 링크 제공

In [1]:
# Python3 샘플 코드 #
# '포항' 키워드를 활용하여 22.01.01~ 23.08.15기간 분실 등록된 자료 불러오기 
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from sklearn import datasets


# STEP1. 데이터 수집 단계 

# 링크 페이지 가져오기; 반복되는 PAIGE loop를 활용하여 수집 (300개 데이터)

In [2]:
# 장소 list 
pl_lst = []
#시간 list
tm_lst = []
#물품 list
sf_lst = []
#고유번호
id_lst = []

In [3]:
url = 'http://apis.data.go.kr/1320000/LostGoodsInfoInqireService/getLostGoodsInfoAccTpNmCstdyPlace'

def make_set(pl_lst, tm_lst, sf_lst,id_lst, url):
    for i in range (1,4):
        #paige = i+10*(i-1)
        params ={'serviceKey' : '+y8zxHxSx9sqT6G4i9gNFkZaqH23k+h51QArnY82ML8D+J3CV1ke+ZAPvIlhtgKUAJPUFi6FaNWgcmehbiwzIA==',
                 'START_YMD' : '20220101',
                 'END_YMD' : '20230701',
                 'numOfRows' : '100', 'LST_PLACE':'포항' , 'pageNo' : str(i)}
        page = requests.get(url, params=params)
        soup =  BeautifulSoup(page.content,'html.parser')
        place = soup.find_all('lstplace')
        [pl_lst.append(x.text) for x in place] #분실장소
        time = soup.find_all('lstymd')
        [tm_lst.append(x.text) for x in time] #분실시간
        stuff = soup.find_all('prdtclnm')
        [sf_lst.append(x.text) for x in stuff] #분실물간
        id = soup.find_all('atcid') 
        [id_lst.append(x.text) for x in id] #고유번호

        

In [4]:
#issue 반복문을 사용하여 paige를 받아오는 경우  'pageNo' : '{i}' >> 해당경우 default값인 1만 반복
#solution: (from. chatGPT) : 문자열 표현하는 방법에 오루가 있어서 발생한것으로 보임 >> 대안 str(i)로 변경하여 기작

In [5]:
make_set(pl_lst, tm_lst, sf_lst,id_lst, url) 

C:\anaconda\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [6]:
len(tm_lst), len(sf_lst), len(pl_lst), len(id_lst)

(300, 300, 300, 300)

In [7]:
#pandas로 데이터 프레임으로 만들기 
lost_df = pd.DataFrame({
    '고유번호': id_lst, 
    '등록날짜': tm_lst,
    '분실장소': pl_lst,
    '물품종류': sf_lst
})

In [8]:
lost_df

,고유번호,등록날짜,분실장소,물품종류
0,L2023081500000396,2023-08-15,남구 대이동 대구은행 뒤 포항정육점 근처,쇼핑백 > 쇼핑백
1,L2023081400000569,2023-07-23,롯데슈퍼 포항점 앞 노상,증명서 > 여권
2,L2023081400000444,2023-08-13,포항방향 외동휴게소,지갑 > 여성용 지갑
3,L2023081300000407,2023-08-12,포항영일대 홍이이자카야 스타벅스사이,귀금속 > 기타
4,L2023081200000690,2023-08-12,포항시 영일대,휴대폰 > 삼성휴대폰
...,...,...,...,...
295,L2022122200000274,2022-12-17,포항 불상지에서 분실,지갑 > 남성용 지갑
296,L2022122000000175,2022-12-08,포항시 기계면 일대 사거리,자동차 > 자동차번호판
297,L2022121900000466,2022-12-12,포항시 도로,자동차 > 자동차번호판
298,L2022121900000430,2022-12-18,격포항 인근,자동차 > 자동차번호판


In [9]:
type(id_lst)

list

# STEP2.  데이터 가공 단계 

In [177]:
# 0. 고유번호 > [, ] split한 이후 index로 넣기 : 중복되는 데이터 있으면 삭제 
# 1. 날짜별로 sort 
# 2. *** 분실장소 별로 (1: 버스, 2: 기차, 3: 택시, 4: 이외공간)
# 3. 물품종류 '>' 기준으로 대분류로 나누기

In [10]:
lost_cpy = lost_df.copy()

# 0. 고유번호 > [, ] split한 이후 index로 넣기 : 중복되는 데이터 있으면 삭제 

In [11]:
# issue :[,] 나누기 
#solution: x.text로 추출하지 않았으므로 해당 값이 list가 아slduTdma; 다시 text형으로 추출하여 해결

In [12]:
#고유번호를 활용하여 중복되는 값 선별 
lost_cpy.groupby('고유번호').count()

,등록날짜,분실장소,물품종류
고유번호,,,
L2022121800000451,1,1,1
L2022121900000430,1,1,1
L2022121900000466,1,1,1
L2022122000000175,1,1,1
L2022122200000274,1,1,1
...,...,...,...
L2023081200000690,1,1,1
L2023081300000407,1,1,1
L2023081400000444,1,1,1


In [13]:
#조건식을 활용하여 데이터 선별 : 중복 값 없음 
result = lost_cpy.groupby('고유번호').filter(lambda group: len(group) > 1)
result

,고유번호,등록날짜,분실장소,물품종류


In [14]:
# '고유번호'를 index 컬럼으로 설정 ; 
lost_cpy.set_index('고유번호', inplace=True) #고유값; encoding 대안: '고유번호'를 index로 변환

In [15]:
lost_cpy

,등록날짜,분실장소,물품종류
고유번호,,,
L2023081500000396,2023-08-15,남구 대이동 대구은행 뒤 포항정육점 근처,쇼핑백 > 쇼핑백
L2023081400000569,2023-07-23,롯데슈퍼 포항점 앞 노상,증명서 > 여권
L2023081400000444,2023-08-13,포항방향 외동휴게소,지갑 > 여성용 지갑
L2023081300000407,2023-08-12,포항영일대 홍이이자카야 스타벅스사이,귀금속 > 기타
L2023081200000690,2023-08-12,포항시 영일대,휴대폰 > 삼성휴대폰
...,...,...,...
L2022122200000274,2022-12-17,포항 불상지에서 분실,지갑 > 남성용 지갑
L2022122000000175,2022-12-08,포항시 기계면 일대 사거리,자동차 > 자동차번호판
L2022121900000466,2022-12-12,포항시 도로,자동차 > 자동차번호판


# 1. 날짜별로 sort 

In [16]:
# 1. 날짜별로 sort 
# sorting 연습 (수입, 보증인수입을 내림차순으로 정렬)
lost_cpy = lost_cpy.sort_values(['등록날짜'], ascending=False)

In [17]:
lost_cpy

,등록날짜,분실장소,물품종류
고유번호,,,
L2023081500000396,2023-08-15,남구 대이동 대구은행 뒤 포항정육점 근처,쇼핑백 > 쇼핑백
L2023081400000444,2023-08-13,포항방향 외동휴게소,지갑 > 여성용 지갑
L2023081300000407,2023-08-12,포항영일대 홍이이자카야 스타벅스사이,귀금속 > 기타
L2023081200000690,2023-08-12,포항시 영일대,휴대폰 > 삼성휴대폰
L2023081100000357,2023-08-10,후포항,자동차 > 자동차번호판
...,...,...,...
L2023011100000425,2022-09-29,대구 포항간 고속도로,자동차 > 자동차번호판
L2023041700000876,2022-09-10,포항시 남구 양포 인근 불상지,자동차 > 자동차번호판
L2023020200000425,2022-04-01,포항시 남구 기림사 가는길,자동차 > 자동차번호판


In [18]:
# 기간이 벋어난 column에 대해서 삭제 
lost_cpy = lost_cpy[:-2]

In [19]:
lost_cpy

,등록날짜,분실장소,물품종류
고유번호,,,
L2023081500000396,2023-08-15,남구 대이동 대구은행 뒤 포항정육점 근처,쇼핑백 > 쇼핑백
L2023081400000444,2023-08-13,포항방향 외동휴게소,지갑 > 여성용 지갑
L2023081300000407,2023-08-12,포항영일대 홍이이자카야 스타벅스사이,귀금속 > 기타
L2023081200000690,2023-08-12,포항시 영일대,휴대폰 > 삼성휴대폰
L2023081100000357,2023-08-10,후포항,자동차 > 자동차번호판
...,...,...,...
L2023071700000702,2022-10-03,후포항 오라오라팬션 250m 근방,휴대폰 > 삼성휴대폰
L2023040600000462,2022-10-01,"포항시 남구 상공로270, 현대오일뱅크 앞",자동차 > 자동차번호판
L2023011100000425,2022-09-29,대구 포항간 고속도로,자동차 > 자동차번호판


# 2.  *** 분실장소 별로 (1: 버스, 2: 기차, 3: 택시, 4: 이외공간)
# >> (버스, 택시, 기차) 키워드로 새로운 데이터를 set을 가지고 이를  

# 2 - 1) '포항'과 연관 없는 지역 filter 

In [20]:
# 데이터 filter 사용; '포항' 관련 없는 지역 filtering 

#'후포항', '대구' 라는 단어가 포함된 데이터를 걸러내기 위해서; .str.contains() 사용

# '대구', '후포항' 이라는 단어가 포함되어있는 경우 
contains_wupohang_or_daegu = lost_cpy['분실장소'].str.contains("후포항|대구")

# 해당 조건을 만족하지 않는 데이터를 새로운 변수에 저장
sub_set_df = lost_cpy[~contains_wupohang_or_daegu]

In [21]:
sub_set_df

,등록날짜,분실장소,물품종류
고유번호,,,
L2023081400000444,2023-08-13,포항방향 외동휴게소,지갑 > 여성용 지갑
L2023081300000407,2023-08-12,포항영일대 홍이이자카야 스타벅스사이,귀금속 > 기타
L2023081200000690,2023-08-12,포항시 영일대,휴대폰 > 삼성휴대폰
L2023081100000326,2023-08-09,포항영일대해수욕장 근처,지갑 > 여성용 지갑
L2023081000000713,2023-08-09,포항영일대해수욕장근처,지갑 > 여성용 지갑
...,...,...,...
L2023020200000428,2022-10-15,포항운전면허시험장 앞 노상,자동차 > 자동차번호판
L2023043000000193,2022-10-07,포항시내 오락실(실내 야구.사격.노래),지갑 > 기타 지갑
L2023040600000462,2022-10-01,"포항시 남구 상공로270, 현대오일뱅크 앞",자동차 > 자동차번호판


# 2- 2) '버스', '기차', '택시' 연관 키워드 filter

In [23]:
# '버스'
# '버스', '버스터미널' 이라는 단어가 포함되어있는 경우 
contain_bus = lost_cpy['분실장소'].str.contains("버스|버스터미널")

# 해당 조건을 만족하는 데이터를 새로운 변수에 저장
bus_df = lost_cpy[contain_bus]
bus_df

,등록날짜,분실장소,물품종류
고유번호,,,
L2023080900000537,2023-08-08,포항시외버스 터미널 맞은편 버스정류장,의류 > 기타의류
L2023080400000632,2023-08-03,포항시 216번 시내버스,지갑 > 남성용 지갑
L2023072100000350,2023-07-21,포항시외버스터미널 부근 택시,지갑 > 여성용 지갑
L2023072100000669,2023-07-21,"305, 900번 버스, 정류장(포항 야구장 건너편)",휴대폰 > 기타통신기기
L2023071600000864,2023-07-16,포항시외버스터미널 건너 버거킹 내,휴대폰 > 삼성휴대폰
L2023070700000708,2023-07-07,죽도시장 지나서 포항 시내가는 길 버스 안 또는 육거리에서 롯데시네마 가는 길,지갑 > 기타 지갑
L2023070400000129,2023-07-03,포항시외버스터미널 앞 어림지,휴대폰 > 삼성휴대폰
L2023062600000493,2023-06-25,포항-창원 행 시외버스 내 (금아여행),지갑 > 남성용 지갑
L2023062100000661,2023-06-21,포항시외버스터미널-해운대시외버스터미널 간 불상,지갑 > 여성용 지갑


# 인코딩 과정 고려하여 '장소' 범주화

In [26]:
#새로 만들 데이터 프레임 bus
bus_cpy = bus_df.copy()

list_bus =[]
for i in range(len(bus_cpy)):
    list_bus.append('버스')

bus_cpy['lost_place'] = list_bus

,등록날짜,분실장소,물품종류
고유번호,,,
L2023080900000537,2023-08-08,포항시외버스 터미널 맞은편 버스정류장,의류 > 기타의류
L2023080400000632,2023-08-03,포항시 216번 시내버스,지갑 > 남성용 지갑
L2023072100000350,2023-07-21,포항시외버스터미널 부근 택시,지갑 > 여성용 지갑
L2023072100000669,2023-07-21,"305, 900번 버스, 정류장(포항 야구장 건너편)",휴대폰 > 기타통신기기
L2023071600000864,2023-07-16,포항시외버스터미널 건너 버거킹 내,휴대폰 > 삼성휴대폰
L2023070700000708,2023-07-07,죽도시장 지나서 포항 시내가는 길 버스 안 또는 육거리에서 롯데시네마 가는 길,지갑 > 기타 지갑
L2023070400000129,2023-07-03,포항시외버스터미널 앞 어림지,휴대폰 > 삼성휴대폰
L2023062600000493,2023-06-25,포항-창원 행 시외버스 내 (금아여행),지갑 > 남성용 지갑
L2023062100000661,2023-06-21,포항시외버스터미널-해운대시외버스터미널 간 불상,지갑 > 여성용 지갑


In [40]:
# '기차'
# 'KTX', '포항역' 이라는 단어가 포함되어있는 경우 
contain_express_tr = lost_cpy['분실장소'].str.contains("ktx|포항역")

# 해당 조건을 만족하는 데이터를 새로운 변수에 저장
tr_df = lost_cpy[contain_express_tr]
tr_df

,등록날짜,분실장소,물품종류
고유번호,,,
L2023080800000119,2023-08-05,포항역ktx 인근 노상,휴대폰 > LG휴대폰
L2023050600000674,2023-05-06,포항역 역내,의류 > 남성의류
L2023042200000184,2023-04-21,포항역,전자기기 > 무선이어폰
L2023041200000436,2023-04-11,아트몰링노브랜드버거~하단역~다대포항역,지갑 > 남성용 지갑
L2023041000000050,2023-04-09,포항시 북구 포항역 택시승강장,휴대폰 > 아이폰
L2023032000000092,2023-03-19,kTX포항역에서 하나로마트 구간,자동차 > 자동차번호판
L2023012400000344,2023-01-24,"포항역 도착 KTX #245, 12호차 5D석",스포츠용품 > 스포츠용품
L2023010300000077,2023-01-01,포항역 주차장,귀금속 > 반지
L2023012700000249,2022-12-31,포항역,증명서 > 신분증


In [42]:
#새로 만들 데이터 프레임 train
tr_cpy = tr_df.copy()

list_train =[]
for i in range(len(tr_cpy)):
    list_train.append('기차')

tr_cpy['lost_place'] = list_train

In [45]:
# '택시'
# '택시' 이라는 단어가 포함되어있는 경우 
contain_taxi = lost_cpy['분실장소'].str.contains("택시")

# 해당 조건을 만족하는 데이터를 새로운 변수에 저장
taxi_df = lost_cpy[contain_taxi]
taxi_df

,등록날짜,분실장소,물품종류
고유번호,,,
L2023072700000787,2023-07-27,포항시 남구 송도동 송도사거리 택시 안,지갑 > 여성용 지갑
L2023072100000350,2023-07-21,포항시외버스터미널 부근 택시,지갑 > 여성용 지갑
L2023062600000822,2023-06-26,포항 쌍용사거리에서 부영3차 이동하는 택시안,휴대폰 > 아이폰
L2023041300000660,2023-04-13,포항여객선터미널에서 영일만 크루즈까지 택시 탑승함.,현금 > 현금
L2023041000000050,2023-04-09,포항시 북구 포항역 택시승강장,휴대폰 > 아이폰
L2023030200000321,2023-03-02,올레시장-서귀포항 택시 안,지갑 > 남성용 지갑


In [46]:
#새로 만들 데이터 프레임 taxi
taxi_cpy = taxi_df.copy()

list_taxi =[]
for i in range(len(taxi_cpy)):
    list_taxi.append('택시')

taxi_cpy['lost_place'] = list_taxi

In [48]:
# '비행기 및 선박'
# '공항' 이라는 단어가 포함되어있는 경우 
contain_port = lost_cpy['분실장소'].str.contains("공항|여객|포항터미널")

# 해당 조건을 만족하는 데이터를 새로운 변수에 저장
port_df = lost_cpy[contain_port]
port_df

,등록날짜,분실장소,물품종류
고유번호,,,
L2023071500000413,2023-07-15,포항터미널 주변,지갑 > 남성용 지갑
L2023061700000753,2023-06-17,포항터미널 or 포항터미널 정류장 or 포항터미널 바로 앞 또는 건너,지갑 > 남성용 지갑
L2023053100000591,2023-05-27,포항터미널 화장실,휴대폰 > 삼성휴대폰
L2023051200000255,2023-05-04,사이판-인천공항-김포-포항중,의류 > 여성의류
L2023041300000660,2023-04-13,포항여객선터미널에서 영일만 크루즈까지 택시 탑승함.,현금 > 현금
L2023032700001044,2023-03-26,포항터미널,악기 > 현악기
L2023032800000131,2023-03-10,목포항- 신안군 신의면 여객선 내,지갑 > 남성용 지갑
L2023031000000815,2023-03-10,포항제철고에서 포항터미널 방향 버스 안,컴퓨터 > 삼성노트북
L2023031700000759,2023-03-05,포항여객선에선 후포사이,증명서 > 신분증


In [55]:
len(port_df)

10

In [49]:
# 공공시설
# 병원, 도서관, 복지센터, 학교, 우체국, 은행, 시험장
contain_official = lost_cpy['분실장소'].str.contains("병원|도서관|복지센터|학교|우체국|은행|시험장|휴게소")

# 해당 조건을 만족하는 데이터를 새로운 변수에 저장
official_df = lost_cpy[contain_official]
official_df

,등록날짜,분실장소,물품종류
고유번호,,,
L2023081500000396,2023-08-15,남구 대이동 대구은행 뒤 포항정육점 근처,쇼핑백 > 쇼핑백
L2023081400000444,2023-08-13,포항방향 외동휴게소,지갑 > 여성용 지갑
L2023071600000729,2023-07-15,영천휴게소 포항방면,지갑 > 남성용 지갑
L2023071200000398,2023-07-12,북포항 우체국 앞,지갑 > 여성용 지갑
L2023060900000701,2023-06-08,경북 포항시 북구 불종로 55 KB국민은행 4거리,휴대폰 > 아이폰
L2023061200001034,2023-06-08,경북 포항시 북구 불종로 55 KB국민은행 4거리 인근,지갑 > 남성용 지갑
L2023053000000899,2023-05-30,포항공과대학교 부근,지갑 > 기타 지갑
L2023052100000698,2023-05-21,서귀포초등학교-서귀포항 노상,가방 > 남성용가방
L2023051900000914,2023-05-19,포항성모병원,귀금속 > 반지


In [56]:
len(official_df)

21

In [72]:
#기타 공공시설 
extra_public_df = pd.concat([port_df, official_df])

list_extra_public =[]
for i in range(len(extra_public_df)):
    list_extra_public.append('기타_공공기관')

extra_public_df['lost_place'] = list_extra_public

In [73]:
len(extra_public_df), len(bus_df), len(tr_df), len(taxi_df)

(31, 34, 10, 6)

In [71]:
extra_public_df

,등록날짜,분실장소,물품종류,lost_place
고유번호,,,,
L2023081400000444,2023-08-13,포항방향 외동휴게소,지갑 > 여성용 지갑,기타_공공기관
L2023071500000413,2023-07-15,포항터미널 주변,지갑 > 남성용 지갑,기타_공공기관
L2023071600000729,2023-07-15,영천휴게소 포항방면,지갑 > 남성용 지갑,기타_공공기관
L2023071200000398,2023-07-12,북포항 우체국 앞,지갑 > 여성용 지갑,기타_공공기관
L2023061700000753,2023-06-17,포항터미널 or 포항터미널 정류장 or 포항터미널 바로 앞 또는 건너,지갑 > 남성용 지갑,기타_공공기관
L2023060900000701,2023-06-08,경북 포항시 북구 불종로 55 KB국민은행 4거리,휴대폰 > 아이폰,기타_공공기관
L2023061200001034,2023-06-08,경북 포항시 북구 불종로 55 KB국민은행 4거리 인근,지갑 > 남성용 지갑,기타_공공기관
L2023053000000899,2023-05-30,포항공과대학교 부근,지갑 > 기타 지갑,기타_공공기관
L2023053100000591,2023-05-27,포항터미널 화장실,휴대폰 > 삼성휴대폰,기타_공공기관


# 공공 시설 <-> 공공시설이 아닌 장소로 분할 

# idea > 사용자 (공공 시설 or 시설이 아닌 지역 선택 

# if 공공 시설: (물품 분류 확인하여 예측) / else : 경찰청 >> 분실 문의 가능 시설 여부
  

In [74]:
#다음 데이터에 해당하지 않는 데이터들의 묶음
not_public_area  =  ~contain_bus & ~contain_express_tr & ~contain_taxi & ~contain_port & ~contain_official

public_df = sub_set_df[~not_public_area] # 공공 시설을 담고 있는 데이터 
not_public_df = sub_set_df[not_public_area] # 공공 시설이 아닌 데이터 

C:\Users\jangs\AppData\Local\Temp\ipykernel_12680\3692291207.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  public_df = sub_set_df[~not_public_area] # 공공 시설을 담고 있는 데이터
C:\Users\jangs\AppData\Local\Temp\ipykernel_12680\3692291207.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_public_df = sub_set_df[not_public_area] # 공공 시설이 아닌 데이터


In [75]:
public_df.count()

등록날짜    71
분실장소    71
물품종류    71
dtype: int64

In [79]:
#가공 데이터 합치기 
public_df =  pd.concat([bus_cpy,tr_cpy,taxi_cpy,extra_public_df]) #중복되는 값 확인하여 삭제 

In [81]:
#공공시설이 아닌 지역 
list_not_public =[]

for i in range(len(not_public_df)):
    list_not_public.append('임의지역')

not_public_df['lost_place'] = list_not_public

C:\Users\jangs\AppData\Local\Temp\ipykernel_12680\578683537.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_public_df['lost_place'] = list_not_public


In [82]:
not_public_df

,등록날짜,분실장소,물품종류,lost_place
고유번호,,,,
L2023081300000407,2023-08-12,포항영일대 홍이이자카야 스타벅스사이,귀금속 > 기타,임의지역
L2023081200000690,2023-08-12,포항시 영일대,휴대폰 > 삼성휴대폰,임의지역
L2023081100000326,2023-08-09,포항영일대해수욕장 근처,지갑 > 여성용 지갑,임의지역
L2023081000000713,2023-08-09,포항영일대해수욕장근처,지갑 > 여성용 지갑,임의지역
L2023080800000671,2023-08-08,격포항 부근,증명서 > 면허증,임의지역
...,...,...,...,...
L2023032200000253,2022-10-20,포항시 북구 덕수동 충혼탑 임구,산업용품 > 기타물품,임의지역
L2023043000000193,2022-10-07,포항시내 오락실(실내 야구.사격.노래),지갑 > 기타 지갑,임의지역
L2023040600000462,2022-10-01,"포항시 남구 상공로270, 현대오일뱅크 앞",자동차 > 자동차번호판,임의지역


# 3. 물품종류 '>' 기준으로 대분류로 나누기 

# 공공 시설(기차, 버스, 택시) 분실한 경우 

In [83]:
public_df

,등록날짜,분실장소,물품종류,lost_place
고유번호,,,,
L2023080900000537,2023-08-08,포항시외버스 터미널 맞은편 버스정류장,의류 > 기타의류,버스
L2023080400000632,2023-08-03,포항시 216번 시내버스,지갑 > 남성용 지갑,버스
L2023072100000350,2023-07-21,포항시외버스터미널 부근 택시,지갑 > 여성용 지갑,버스
L2023072100000669,2023-07-21,"305, 900번 버스, 정류장(포항 야구장 건너편)",휴대폰 > 기타통신기기,버스
L2023071600000864,2023-07-16,포항시외버스터미널 건너 버거킹 내,휴대폰 > 삼성휴대폰,버스
...,...,...,...,...
L2023021300001000,2023-02-13,포항공과대학교,휴대폰 > 삼성휴대폰,기타_공공기관
L2023031000000460,2023-01-25,포항-울산 고속도로 외동휴게소 근처,자동차 > 자동차번호판,기타_공공기관
L2023012000000319,2023-01-19,대구포항간 고속도로 대구방향 영천휴게소 2키로전,자동차 > 자동차번호판,기타_공공기관


# 3. 물품종류 '>' 기준으로 대분류로 나누기

In [84]:
stuff_type = public_df['물품종류']

In [85]:
#물품 대분류와 소분류 나누기 
for i in range(len(public_df)):
    stuff_type[i] = stuff_type[i].split('>')
    #문자열 대분류 , 소분류 나누기 (1)
stuff_type

고유번호
L2023080900000537       [의류 ,  기타의류]
L2023080400000632     [지갑 ,  남성용 지갑]
L2023072100000350     [지갑 ,  여성용 지갑]
L2023072100000669    [휴대폰 ,  기타통신기기]
L2023071600000864     [휴대폰 ,  삼성휴대폰]
                          ...       
L2023021300001000     [휴대폰 ,  삼성휴대폰]
L2023031000000460    [자동차 ,  자동차번호판]
L2023012000000319    [자동차 ,  자동차번호판]
L2022122200000673     [휴대폰 ,  삼성휴대폰]
L2023020200000428    [자동차 ,  자동차번호판]
Name: 물품종류, Length: 81, dtype: object

In [86]:
len(stuff_type)

81

In [87]:
# 반복문을 통해 자료 리스트 만들기 (pageNo 1~7) 22~23년 데이터 
type_lst = [] 
for i in range(len(public_df)):
    type_lst.append(stuff_type[i][0]) 
len(type_lst)

81

In [108]:
# 새로운 컬럼 생성 
public_df['type_stuff'] = type_lst
public_df

,등록날짜,분실장소,물품종류,lost_place,type,type_public,type_stuff
고유번호,,,,,,,
L2023080900000537,2023-08-08,포항시외버스 터미널 맞은편 버스정류장,"[의류 , 기타의류]",버스,의류,yes,의류
L2023080400000632,2023-08-03,포항시 216번 시내버스,"[지갑 , 남성용 지갑]",버스,지갑,yes,지갑
L2023072100000350,2023-07-21,포항시외버스터미널 부근 택시,"[지갑 , 여성용 지갑]",버스,지갑,yes,지갑
L2023072100000669,2023-07-21,"305, 900번 버스, 정류장(포항 야구장 건너편)","[휴대폰 , 기타통신기기]",버스,휴대폰,yes,휴대폰
L2023071600000864,2023-07-16,포항시외버스터미널 건너 버거킹 내,"[휴대폰 , 삼성휴대폰]",버스,휴대폰,yes,휴대폰
...,...,...,...,...,...,...,...
L2023021300001000,2023-02-13,포항공과대학교,"[휴대폰 , 삼성휴대폰]",기타_공공기관,휴대폰,yes,휴대폰
L2023031000000460,2023-01-25,포항-울산 고속도로 외동휴게소 근처,"[자동차 , 자동차번호판]",기타_공공기관,자동차,yes,자동차
L2023012000000319,2023-01-19,대구포항간 고속도로 대구방향 영천휴게소 2키로전,"[자동차 , 자동차번호판]",기타_공공기관,자동차,yes,자동차


In [107]:
public_df.isna().sum()

등록날짜           0
분실장소           0
물품종류           0
lost_place     0
type           0
type_public    0
dtype: int64

# 공공 시설이 아닌 장소에서 분실한 경우 

In [89]:
stuff1_type = not_public_df['물품종류']
stuff1_type

고유번호
L2023081300000407        귀금속 > 기타
L2023081200000690     휴대폰 > 삼성휴대폰
L2023081100000326     지갑 > 여성용 지갑
L2023081000000713     지갑 > 여성용 지갑
L2023080800000671       증명서 > 면허증
                         ...     
L2023032200000253     산업용품 > 기타물품
L2023043000000193      지갑 > 기타 지갑
L2023040600000462    자동차 > 자동차번호판
L2023041700000876    자동차 > 자동차번호판
L2023020200000425    자동차 > 자동차번호판
Name: 물품종류, Length: 213, dtype: object

In [90]:
#물품 대분류와 소분류 나누기 
for i in range(len(not_public_df)):
    stuff1_type[i] = stuff1_type[i].split('>')
    #문자열 대분류 , 소분류 나누기 (1)
stuff1_type

C:\Users\jangs\AppData\Local\Temp\ipykernel_12680\3320128693.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stuff1_type[i] = stuff1_type[i].split('>')


고유번호
L2023081300000407        [귀금속 ,  기타]
L2023081200000690     [휴대폰 ,  삼성휴대폰]
L2023081100000326     [지갑 ,  여성용 지갑]
L2023081000000713     [지갑 ,  여성용 지갑]
L2023080800000671       [증명서 ,  면허증]
                          ...       
L2023032200000253     [산업용품 ,  기타물품]
L2023043000000193      [지갑 ,  기타 지갑]
L2023040600000462    [자동차 ,  자동차번호판]
L2023041700000876    [자동차 ,  자동차번호판]
L2023020200000425    [자동차 ,  자동차번호판]
Name: 물품종류, Length: 213, dtype: object

In [91]:
# 반복문을 통해 자료 리스트 만들기 (pageNo 1~7) 22~23년 데이터 
type1_lst = [] 
for i in range(len(not_public_df)):
    type1_lst.append(stuff1_type[i][0]) 
type1_lst

['귀금속 ',
 '휴대폰 ',
 '지갑 ',
 '지갑 ',
 '증명서 ',
 '가방 ',
 '자동차 ',
 '자동차 ',
 '휴대폰 ',
 '귀금속 ',
 '지갑 ',
 '자동차 ',
 '가방 ',
 '지갑 ',
 '가방 ',
 '자동차 ',
 '스포츠용품 ',
 '증명서 ',
 '휴대폰 ',
 '휴대폰 ',
 '자동차 ',
 '지갑 ',
 '지갑 ',
 '휴대폰 ',
 '자동차 ',
 '자동차 ',
 '휴대폰 ',
 '유가증권 ',
 '지갑 ',
 '지갑 ',
 '휴대폰 ',
 '지갑 ',
 '자동차 ',
 '자동차 ',
 '전자기기 ',
 '귀금속 ',
 '자동차 ',
 '지갑 ',
 '증명서 ',
 '지갑 ',
 '지갑 ',
 '기타물품 ',
 '자동차 ',
 '자동차 ',
 '기타물품 ',
 '자동차 ',
 '지갑 ',
 '휴대폰 ',
 '자동차 ',
 '자동차 ',
 '전자기기 ',
 '지갑 ',
 '자동차 ',
 '스포츠용품 ',
 '지갑 ',
 '지갑 ',
 '휴대폰 ',
 '휴대폰 ',
 '자동차 ',
 '귀금속 ',
 '자동차 ',
 '지갑 ',
 '기타물품 ',
 '휴대폰 ',
 '자동차 ',
 '전자기기 ',
 '지갑 ',
 '지갑 ',
 '휴대폰 ',
 '스포츠용품 ',
 '지갑 ',
 '지갑 ',
 '지갑 ',
 '자동차 ',
 '유가증권 ',
 '자동차 ',
 '휴대폰 ',
 '자동차 ',
 '자동차 ',
 '서류 ',
 '지갑 ',
 '자동차 ',
 '자동차 ',
 '기타물품 ',
 '자동차 ',
 '가방 ',
 '증명서 ',
 '휴대폰 ',
 '자동차 ',
 '휴대폰 ',
 '카드 ',
 '자동차 ',
 '자동차 ',
 '지갑 ',
 '지갑 ',
 '지갑 ',
 '자동차 ',
 '휴대폰 ',
 '지갑 ',
 '전자기기 ',
 '휴대폰 ',
 '가방 ',
 '가방 ',
 '자동차 ',
 '휴대폰 ',
 '휴대폰 ',
 '증명서 ',
 '지갑 ',
 '지갑 ',
 '자동차 ',
 '귀금속 ',
 '지갑 ',
 '지갑 ',
 '자동차 

In [92]:
# 새로운 컬럼 생성 
not_public_df['type_stuff'] = type1_lst
not_public_df

C:\Users\jangs\AppData\Local\Temp\ipykernel_12680\1090457556.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_public_df['type_stuff'] = type1_lst


,등록날짜,분실장소,물품종류,lost_place,type_stuff
고유번호,,,,,
L2023081300000407,2023-08-12,포항영일대 홍이이자카야 스타벅스사이,"[귀금속 , 기타]",임의지역,귀금속
L2023081200000690,2023-08-12,포항시 영일대,"[휴대폰 , 삼성휴대폰]",임의지역,휴대폰
L2023081100000326,2023-08-09,포항영일대해수욕장 근처,"[지갑 , 여성용 지갑]",임의지역,지갑
L2023081000000713,2023-08-09,포항영일대해수욕장근처,"[지갑 , 여성용 지갑]",임의지역,지갑
L2023080800000671,2023-08-08,격포항 부근,"[증명서 , 면허증]",임의지역,증명서
...,...,...,...,...,...
L2023032200000253,2022-10-20,포항시 북구 덕수동 충혼탑 임구,"[산업용품 , 기타물품]",임의지역,산업용품
L2023043000000193,2022-10-07,포항시내 오락실(실내 야구.사격.노래),"[지갑 , 기타 지갑]",임의지역,지갑
L2023040600000462,2022-10-01,"포항시 남구 상공로270, 현대오일뱅크 앞","[자동차 , 자동차번호판]",임의지역,자동차


# 데이터 분실 유형 (공공장소 여부)

In [93]:
type_public = []
for i in range(len(public_df)):
    type_public.append('yes')
public_df['type_public'] = type_public

In [94]:
public_df

,등록날짜,분실장소,물품종류,lost_place,type,type_public
고유번호,,,,,,
L2023080900000537,2023-08-08,포항시외버스 터미널 맞은편 버스정류장,"[의류 , 기타의류]",버스,의류,yes
L2023080400000632,2023-08-03,포항시 216번 시내버스,"[지갑 , 남성용 지갑]",버스,지갑,yes
L2023072100000350,2023-07-21,포항시외버스터미널 부근 택시,"[지갑 , 여성용 지갑]",버스,지갑,yes
L2023072100000669,2023-07-21,"305, 900번 버스, 정류장(포항 야구장 건너편)","[휴대폰 , 기타통신기기]",버스,휴대폰,yes
L2023071600000864,2023-07-16,포항시외버스터미널 건너 버거킹 내,"[휴대폰 , 삼성휴대폰]",버스,휴대폰,yes
...,...,...,...,...,...,...
L2023021300001000,2023-02-13,포항공과대학교,"[휴대폰 , 삼성휴대폰]",기타_공공기관,휴대폰,yes
L2023031000000460,2023-01-25,포항-울산 고속도로 외동휴게소 근처,"[자동차 , 자동차번호판]",기타_공공기관,자동차,yes
L2023012000000319,2023-01-19,대구포항간 고속도로 대구방향 영천휴게소 2키로전,"[자동차 , 자동차번호판]",기타_공공기관,자동차,yes


In [95]:
type_non_public = []
for i in range(len(not_public_df)):
    type_non_public.append('no')
not_public_df['type_public']  = type_non_public

C:\Users\jangs\AppData\Local\Temp\ipykernel_12680\2417047683.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_public_df['type_public']  = type_non_public


In [96]:
not_public_df.drop('물품종류',axis=1) #물품정보 중복되는 내용 삭제 

,등록날짜,분실장소,lost_place,type_stuff,type_public
고유번호,,,,,
L2023081300000407,2023-08-12,포항영일대 홍이이자카야 스타벅스사이,임의지역,귀금속,no
L2023081200000690,2023-08-12,포항시 영일대,임의지역,휴대폰,no
L2023081100000326,2023-08-09,포항영일대해수욕장 근처,임의지역,지갑,no
L2023081000000713,2023-08-09,포항영일대해수욕장근처,임의지역,지갑,no
L2023080800000671,2023-08-08,격포항 부근,임의지역,증명서,no
...,...,...,...,...,...
L2023032200000253,2022-10-20,포항시 북구 덕수동 충혼탑 임구,임의지역,산업용품,no
L2023043000000193,2022-10-07,포항시내 오락실(실내 야구.사격.노래),임의지역,지갑,no
L2023040600000462,2022-10-01,"포항시 남구 상공로270, 현대오일뱅크 앞",임의지역,자동차,no


In [102]:
not_public_df.isna().sum()

등록날짜           0
분실장소           0
물품종류           0
lost_place     0
type_stuff     0
type_public    0
dtype: int64

In [103]:
public_df.isna().sum()

등록날짜           0
분실장소           0
물품종류           0
lost_place     0
type           0
type_public    0
dtype: int64

# 분실장소 > 분류 단일화 (보류)

#공공: 버스, 택시, 기차, (항구,비행기)선착장, 공공시설
#비공식 기관: 인도, 해수욕장, 주유소, 시장 shuffle 하여 설정 

# 데이터 합치기 (concat사용)

In [112]:
total_set = pd.concat([not_public_df, public_df]) #2 데이터 프레임 합치기 
total_set = total_set.sort_values(['등록날짜'], ascending=False) #정렬
total_set

,등록날짜,분실장소,물품종류,lost_place,type_stuff,type_public
고유번호,,,,,,
L2023081500000396,2023-08-15,남구 대이동 대구은행 뒤 포항정육점 근처,"[쇼핑백 , 쇼핑백]",기타_공공기관,쇼핑백,yes
L2023081400000444,2023-08-13,포항방향 외동휴게소,"[지갑 , 여성용 지갑]",기타_공공기관,지갑,yes
L2023081300000407,2023-08-12,포항영일대 홍이이자카야 스타벅스사이,"[귀금속 , 기타]",임의지역,귀금속,no
L2023081200000690,2023-08-12,포항시 영일대,"[휴대폰 , 삼성휴대폰]",임의지역,휴대폰,no
L2023081100000326,2023-08-09,포항영일대해수욕장 근처,"[지갑 , 여성용 지갑]",임의지역,지갑,no
...,...,...,...,...,...,...
L2023020200000428,2022-10-15,포항운전면허시험장 앞 노상,"[자동차 , 자동차번호판]",기타_공공기관,자동차,yes
L2023043000000193,2022-10-07,포항시내 오락실(실내 야구.사격.노래),"[지갑 , 기타 지갑]",임의지역,지갑,no
L2023040600000462,2022-10-01,"포항시 남구 상공로270, 현대오일뱅크 앞","[자동차 , 자동차번호판]",임의지역,자동차,no


# 중복되는 column삭제 및 encoding 

In [113]:
# '물품종류 ' <-> 'type' 중복이 됨으로 삭제 
total_set = total_set.drop('물품종류',axis=1) 

In [114]:
total_set.dtypes

등록날짜           object
분실장소           object
lost_place     object
type_stuff     object
type_public    object
dtype: object

# excel 파일로 저장 

In [115]:
total_set.to_excel('포항시_분실물데이터_project.xlsx')

In [ ]:
# 구체적인 상품 분류

In [252]:
public_df.groupby('type').count()

,등록날짜,분실장소,물품종류
type,,,
가방,3,3,3
귀금속,4,4,4
스포츠용품,1,1,1
악기,1,1,1
유류품,1,1,1
의류,4,4,4
자동차,7,7,7
전자기기,4,4,4
증명서,6,6,6


In [ ]:
# 날짜, 
#날짜, 위치(포항시 북구/남구), 물품 분류, 물품 색상 + target: 대중교통 시설 인지 아닌지 (맞음: 1, 아님: 0)

In [ ]:
# 분실물 발생하였을때 대중교통 시설인 경우와 아닌경우를 나누어, 대중교통인 시설인 경우 해당 시설의 전화번호및 문의 링크 연결, 아닌 경우 경찰청 분실물 링크 연결  
# GOAL: 사용자가 분실 물품을 입ㄹ력하면 잃어ㅂㅈ벌힌 장소를 예측, -> 사용자 y/n 입력 받아 -> IF: y:대응하는 전화번호 , n: 사용자가 직접 장소 입력, -> 장소 해당하는 링크 연결
# y: : 대중교통에서 분실항 경우 
# x:  나머지 행